In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("churn_pred").getOrCreate()

In [ ]:
df = spark.read.csv("../input/generalassemblywelcome5k/customer_churn.csv",header=True , inferSchema=True)

In [ ]:
df.printSchema()

In [ ]:
df.describe().show()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=[
#  'Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
#  'Onboard_date',
#  'Location',
#  'Company',
#  'Churn'
 ],
 outputCol="features")

In [ ]:
output = assembler.transform(df)

In [ ]:
df_final = output.select("features","churn")

In [ ]:
df_final.show()

In [ ]:
train , test = df_final.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(labelCol="churn")

In [ ]:
lrm = lr.fit(train)

In [ ]:
lrm.summary

In [ ]:
lrm.summary.accuracy

In [ ]:
lrm.summary.areaUnderROC

In [ ]:
lrm.summary.predictions.show()

In [ ]:
lrm.summary.predictions.describe().show()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
pred_labels = lrm.evaluate(test)

In [ ]:
pred_labels.predictions.show()

In [ ]:
eval = BinaryClassificationEvaluator(rawPredictionCol="prediction" , labelCol='churn')

In [ ]:
auc = eval.evaluate(pred_labels.predictions)

In [ ]:
auc